In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gmaps
import pyproj
# from pyproj import Proj, transform

In [2]:
gmaps.configure(api_key='') # Fill in with your API key

In [3]:
# Load Sampled Data
df=pd.read_csv("Simplified_Citation_After_2015July_to_2019.csv",low_memory=False)
df.head(10)

,Unnamed: 0,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
0,28,4361326712,2019/09/17 12:00:00 AM,1205.0,NaN,NaN,CA,NaN,TOYT,PA,GY,309 WINDWARD AVE,00163,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.419487e+06,1.818861e+06
1,112,4361286823,2019/09/17 12:00:00 AM,853.0,NaN,NaN,CA,NaN,TOYT,PA,WT,934 84TH ST W,00553,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.473690e+06,1.808839e+06
2,224,4361159515,2019/09/17 12:00:00 AM,833.0,NaN,NaN,CA,NaN,NISS,PA,WT,2601 MONMOUTH AVE,00536,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.475329e+06,1.834194e+06
3,473,4361237226,2019/09/17 12:00:00 AM,1015.0,NaN,NaN,CA,NaN,HOND,PA,RD,615 HAMPTON DR,00133,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.417752e+06,1.820642e+06
4,592,4360129196,2019/09/17 12:00:00 AM,1007.0,NaN,NaN,CA,NaN,VOLV,PA,SL,1346 MCCADDEN PL N,00487,54.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.459495e+06,1.857205e+06
5,618,4360059804,2019/09/17 12:00:00 AM,1032.0,NaN,NaN,CA,NaN,CHEV,PA,BK,250 CORONADO ST S,00452,54.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.478569e+06,1.848477e+06
6,679,4359986643,2019/09/17 12:00:00 AM,1054.0,NaN,NaN,CA,NaN,HOND,PA,WT,15211 KITTRIDGE ST,350A,53.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.421280e+06,1.891956e+06
7,694,4359985136,2019/09/17 12:00:00 AM,1057.0,NaN,NaN,CA,NaN,TOYT,PA,BL,2131 CANYON DR,00402,54.0,80.73.2,EXCEED 72HRS-ST,68.0,6.465731e+06,1.862324e+06
8,824,4359867912,2019/09/17 12:00:00 AM,1028.0,NaN,NaN,CA,NaN,MERC,PA,SL,1222 POINT VIEW ST,00143,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.449820e+06,1.842157e+06
9,897,4359725355,2019/09/17 12:00:00 AM,138.0,NaN,NaN,CA,NaN,JEEP,PA,GY,1000 39TH ST W,00500,55.0,80.56E4+,RED ZONE,93.0,6.473318e+06,1.828144e+06


In [4]:
df['Longitude'].value_counts()

9.999900e+04    11416
1.819688e+06       98
1.859071e+06       87
1.882602e+06       64
1.849114e+06       49
                ...  
1.843271e+06        1
1.845315e+06        1
1.909664e+06        1
1.835485e+06        1
1.857892e+06        1
Name: Longitude, Length: 50929, dtype: int64

In [5]:
len(df[(df['Latitude'] == 99999) | (df['Longitude'] == 99999)])/len(df)

0.13038810334193754

In [6]:
df_cleaned = df[(df['Latitude'] != 99999) | (df['Longitude'] != 99999)].copy()

In [7]:
df_cleaned[['Latitude', 'Longitude']].describe()

,Latitude,Longitude
count,7.613800e+04,7.613800e+04
mean,6.455758e+06,1.848533e+06
std,2.734972e+04,2.866901e+04
min,6.360390e+06,1.715644e+06
25%,6.431560e+06,1.837057e+06
50%,6.458808e+06,1.845113e+06
75%,6.478079e+06,1.859516e+06
max,6.513150e+06,1.941866e+06


In [8]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76138 entries, 0 to 87553
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             76138 non-null  int64  
 1   Ticket number          76138 non-null  int64  
 2   Issue Date             76138 non-null  object 
 3   Issue time             76127 non-null  float64
 4   Meter Id               21218 non-null  object 
 5   Marked Time            2556 non-null   float64
 6   RP State Plate         76137 non-null  object 
 7   VIN                    0 non-null      float64
 8   Make                   76107 non-null  object 
 9   Body Style             76098 non-null  object 
 10  Color                  76126 non-null  object 
 11  Location               76138 non-null  object 
 12  Route                  75807 non-null  object 
 13  Agency                 76138 non-null  float64
 14  Violation code         76138 non-null  object 
 15  Vi

In [47]:
inProj = Proj('epsg:2229', preserve_units=True) # make sure unit preserve 
outProj = Proj('epsg:4326')
y_1,x_1 = transform(inProj,outProj,df_cleaned["Latitude"].values,df_cleaned["Longitude"].values)

In [49]:
df_cleaned["Latitude_converted"] = y_1
df_cleaned["Longitude_converted"] = x_1
df_cleaned.head()

,Unnamed: 0,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,VIN,Make,Body Style,...,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude,Longitude_converted,Latitude_converted
0,28,4361326712,2019/09/17 12:00:00 AM,1205.0,NaN,NaN,CA,NaN,TOYT,PA,...,309 WINDWARD AVE,00163,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.419487e+06,1.818861e+06,-136.225808,45.373964
1,112,4361286823,2019/09/17 12:00:00 AM,853.0,NaN,NaN,CA,NaN,TOYT,PA,...,934 84TH ST W,00553,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.473690e+06,1.808839e+06,-136.300985,45.512650
2,224,4361159515,2019/09/17 12:00:00 AM,833.0,NaN,NaN,CA,NaN,NISS,PA,...,2601 MONMOUTH AVE,00536,55.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.475329e+06,1.834194e+06,-136.206612,45.529318
3,473,4361237226,2019/09/17 12:00:00 AM,1015.0,NaN,NaN,CA,NaN,HOND,PA,...,615 HAMPTON DR,00133,51.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.417752e+06,1.820642e+06,-136.217917,45.370232
4,592,4360129196,2019/09/17 12:00:00 AM,1007.0,NaN,NaN,CA,NaN,VOLV,PA,...,1346 MCCADDEN PL N,00487,54.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.459495e+06,1.857205e+06,-136.109033,45.498513


In [50]:
df_cleaned.isnull().sum()


Unnamed: 0                   0
Ticket number                0
Issue Date                   0
Issue time                  11
Meter Id                 54920
Marked Time              73582
RP State Plate               1
VIN                      76138
Make                        31
Body Style                  40
Color                       12
Location                     0
Route                      331
Agency                       0
Violation code               0
Violation Description        2
Fine amount                 23
Latitude                     0
Longitude                    0
Longitude_converted          0
Latitude_converted           0
dtype: int64

In [52]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76138 entries, 0 to 87553
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             76138 non-null  int64  
 1   Ticket number          76138 non-null  int64  
 2   Issue Date             76138 non-null  object 
 3   Issue time             76127 non-null  float64
 4   Meter Id               21218 non-null  object 
 5   Marked Time            2556 non-null   float64
 6   RP State Plate         76137 non-null  object 
 7   VIN                    0 non-null      float64
 8   Make                   76107 non-null  object 
 9   Body Style             76098 non-null  object 
 10  Color                  76126 non-null  object 
 11  Location               76138 non-null  object 
 12  Route                  75807 non-null  object 
 13  Agency                 76138 non-null  float64
 14  Violation code         76138 non-null  object 
 15  Vi

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6782765 entries, 0 to 7570560
Data columns (total 20 columns):
Ticket number            object
Issue Date               object
Issue time               float64
Meter Id                 object
Marked Time              float64
RP State Plate           object
VIN                      float64
Make                     object
Body Style               object
Color                    object
Location                 object
Route                    object
Agency                   float64
Violation code           object
Violation Description    object
Fine amount              object
Latitude                 float64
Longitude                float64
Latitude_1               float64
Longitude_1              float64
dtypes: float64(8), object(12)
memory usage: 1.1+ GB


In [ ]:
#gmaps.configure(api_key='')
#df = df[(df["Violation Description"] == "NO PARK/STREET CLEAN") | (df["Violation Description"] == "METER EXP.")]
#locations = df.iloc[:100][["Latitude_1","Longitude_1"]]
#fig = gmaps.figure()
#fig.add_layer(gmaps.heatmap_layer(locations))
#fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
#gmaps.configure(api_key='') 
#locations = df.iloc[:1000000][["Latitude_1","Longitude_1"]]
#fig = gmaps.figure()
#fig.add_layer(gmaps.heatmap_layer(locations))
#fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
'''gmaps.configure(api_key='')
issue_time_morning = (df["Issue time"] > 800) & (df["Issue time"] < 1200)
few_data = df[issue_time_morning][df["Violation Description"] == "RED ZONE"]
locations = few_data.ilo[:10000][["Latitude_1","Longitude_1"]]
fig = gmaps.figure()
layer = gmaps.symbol_layer(locations, fill_color = "red", stroke_color = "red", scale = 1)
fig.add_layer(layer)
fig'''

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


Figure(layout=FigureLayout(height='420px'))

In [ ]:
#picked 100000 samples since my computer cannot process more data than that 
gmaps.configure(api_key='') # type in your own API 
morning_time = (df["Issue time"] > 600) & (df["Issue time"] <= 1200) & (df.iloc[:100000]["Violation Description"] == "RED ZONE")
morning_time_df = df[morning_time][["Latitude_1","Longitude_1"]]

afternoon_time = (df["Issue time"] > 1200) & (df["Issue time"] <= 1800) & (df.iloc[:100000]["Violation Description"] == "RED ZONE")
afternoon_time_df = df[afternoon_time][["Latitude_1","Longitude_1"]]

evening_time = (df["Issue time"] > 1800)  & (df.iloc[:100000]["Violation Description"] == "RED ZONE")
evening_time_df = df[evening_time][["Latitude_1","Longitude_1"]]

fig = gmaps.figure()
morning_layer = gmaps.symbol_layer(morning_time_df, fill_color = "red", stroke_color = "red", scale = 1)
afternoon_layer = gmaps.symbol_layer(afternoon_time_df, fill_color = "green", stroke_color = "green", scale = 1)
evening_layer = gmaps.symbol_layer(evening_time_df, fill_color = "blue", stroke_color = "blue", scale = 1)
fig.add_layer(morning_layer)
fig.add_layer(afternoon_layer)
fig.add_layer(evening_layer)
fig

In [59]:
#picked 100000 samples since my computer cannot process more data than that 
gmaps.configure(api_key='') # type in your own API 
morning_time = (df_cleaned["Issue time"] > 600) & (df_cleaned["Issue time"] <= 1200) & (df_cleaned.iloc[:100000]["Violation Description"] == "RED ZONE")
morning_time_df = df_cleaned[morning_time][["Latitude_converted","Longitude_converted"]]

afternoon_time = (df_cleaned["Issue time"] > 1200) & (df_cleaned["Issue time"] <= 1800) & (df_cleaned.iloc[:100000]["Violation Description"] == "RED ZONE")
afternoon_time_df = df_cleaned[afternoon_time][["Latitude_converted","Longitude_converted"]]

evening_time = (df_cleaned["Issue time"] > 1800)  & (df_cleaned.iloc[:100000]["Violation Description"] == "RED ZONE")
evening_time_df = df_cleaned[evening_time][["Latitude_converted","Longitude_converted"]]

fig = gmaps.figure()
morning_layer = gmaps.symbol_layer(morning_time_df, fill_color = "red", stroke_color = "red", scale = 1)
afternoon_layer = gmaps.symbol_layer(afternoon_time_df, fill_color = "green", stroke_color = "green", scale = 1)
evening_layer = gmaps.symbol_layer(evening_time_df, fill_color = "blue", stroke_color = "blue", scale = 1)
fig.add_layer(morning_layer)
fig.add_layer(afternoon_layer)
fig.add_layer(evening_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [58]:
%matplotlib 

Using matplotlib backend: Qt5Agg
